# Crystal Cascade
A Sierra Nevada/Little Creatures inspired American Pale Ale
Pretty much the same as this [Basic Cascade](https://www.brewtoad.com/recipes/basic-cascade-584c84) I came accross after I'd started this recipe.

## Recipe

### Malt Bill:
 - Pilsner Malt - 2 Row (?EBC)
 - Australian Ale Malt - 2 Row (?EBC)
 - Crystal Malt (?EBC)

### Hops:
 - Cascade (?Alpha Acids)

Make a widget window for all the inputs

In [1]:
from ipywidgets import interact, fixed
import ipywidgets as widgets

In [2]:
def make_widget(value, description):
    style = {'description_width' : '50%'}
    return widgets.FloatText(value=value, description=description, style=style)


batch_size_box = make_widget(5, 'Batch Size (L) =')
malt_mass_box = make_widget(1, 'Total Malt Mass (kg) =')

words = ['correct', 'horse', 'battery', 'staple']
items = [widgets.FloatText(description=w) for w in words]
left_box = widgets.VBox([batch_size_box,
                         malt_mass_box])
mid_box = widgets.VBox([items[0],
                        items[1]])
right_box = widgets.VBox([items[2],
                          items[3]])
widgets.HBox([left_box, mid_box, right_box])

### Calculate Water amounts
Need functions to take in batch size and work out the amounts all the way to strike water, inculding assumptions for losses etc.
Calculations and definitions from [Mash Hacks](https://mashhacks.com/how-to-calculate-water-volumes-for-brewing/):

- **Strike Water** - The starting amount of hot water (usually between 60C and 80C) that is used for the mash (adding grain to water).
- **Grain Absorption** - The amount of water absorbed by the grain.
- **First Runnings** - The amount of wort (sugary water) that is collected from the mash tun after your mash has finished.
- **Sparge Water** - Water that is added to the mash tun after mashing to rinse the grain of any left over sugars.
- **Pre-Boil Volume** - The amount of wort (sugary water) that is in your kettle before you start your boil.
- **Evaporation Rate** - The amount of water that is boiled off. Usually in Litres per Hour (or Gal/hr).
- **Post-Boil Volume** - The amount of wort left after you finish the boil.
- **Kettle Loss** - The amount of wort (if any) left at the bottom of the kettle that was not transferred to the fermenter.
- **Fermenter Volume** - The amount of wort you were able to get out of the kettle and put into the fermenter.
- **Trub Loss** aka *Fermenter loss* - The amount of beer (if any) at the bottom of the fermenter that was not transferred into bottles or kegs.
- **Batch Size** - The amount of beer that you were able to get out of the fermenter and put into bottles or kegs.

Batch Size: 20L (5 gal)

Fermenter Volume = Batch Size + Trub Loss 
Fermenter Volume = 20L (5 gal) + 1L (0.25 gal) = 21L (5.25 gal)

Post-Boil Volume = Fermenter Volume + Kettle Loss 
Post-Boil Volume = 21L (5.25 gal) + 0L (0.00 gal) = 21L (5.25 gal)

Evaporation rates vary quite wildly from system to system, so you'll only know your rate once you've tried. A nice guess would be 5L/hr (1.25 gal/hr), but remember to measure this yourself! We'll use 3L/hr (0.75 gal/hr) for this example.

Pre-Boil Volume = Post-Boil Volume + (Evaporation Rate x Boil Length in Hours) 
Pre-Boil Volume = 21L (5.25 gal) + (3L/hr (0.75 gal/hr) x 1) = 24L (6 gal)

First we need to figure out how much strike water we'll need. The amount of strike water you need depends on how much grain you have in your grain bill (aka grist). This one is more of a guide towards best practice mash thickness, but you can alter to suit your needs. In this example we'll have 6kg grain, or 14 pounds for imperial.

Metric 
Strike Water (L) = Grain Weight (kg) x 3 
Strike Water = 6 x 3 = 18L

Imperial 
Strike Water (quarts) = Grain Weight (pounds) x 1.25 
Strike Water = 14 x 1.25 = 17.5 quarts = ~4.5 gal

Metric 
Grain Absorption = 1 L/Kg 
Grain Absorption = 6L

Imperial 
Grain Absorption = 1 pint/pound 
Grain Absorption = 14 pints = ~1.5 gal

To calculate our first runnings, simply remove the grain absorption volume from your strike volume.

First Runnings = Strike Volume - Grain Absorption 
First Runnings = 18L (4.5 gal) - 6L (1.5 gal) = 12L (3 gal)

Sparge Water = Pre-Boil Volume - First Runnings 
Sparge Water = 24L (6 gal) - 12L (3 gal) = 12L (3 gal)

In [3]:
batch_size = batch_size_box.value
print(batch_size, 'L')

5.0 L


In [4]:
# This will be a module (file)
class constants(object):
    # Approximate rate of evaporation at boil (L/hr)
    evap_rate = 5
    # Approximate loss of volume due to grain absorption
    grain_disp = ????????????
    # Efficiency of getting sugars from malt
    eff = ?????????

SyntaxError: invalid syntax (<ipython-input-4-2db0755751a1>, line 4)

In [ ]:
def strike_water():
    pass
def pre_boil_volume():
    pass
def post_boil_volume():
    return pre_boil_volume() - evaporation(time)
def evaporation(time=1):
    """ Returns evaporation (L) for a given boil time (hrs) """
    return time * constants.evap_rate

### Calculate Expected Original Gravity

In [ ]:
malt_mass = malt_mass_box.value
print(batch_size, 'L;', malt_mass, 'kg')